In [5]:
%run ./spark-instance.ipynb

SparkConf created
Started SparkSession
Spark version 3.3.1


In [27]:
clean_spark()  # run by EOB

CLEANING SPARK INSTANCE...


In [3]:
from datetime import datetime

import pyspark.sql.functions as psf
import pyspark.sql.types as pst

In [7]:
date = datetime(2024, 6, 25) 
censys_df = spark.read.format("parquet").load(
    f"s3a://censys/dataset=universal-internet-dataset-v2-ipv4/format=parquet/year={date.year}/month={date.month:02d}/day={date.day:02d}"
)

In [10]:
win_server_2016_jarms = [
    "2ad2ad16d2ad2ad22c2ad2ad2ad2adfd9c9d14e4f4f67f94f0359f8b28f532",
    "2ad2ad0002ad2ad0002ad2ad2ad2adf9fdf4eeac344e8b5003264da73585be",
    "2ad2ad0002ad2ad08c2ad2ad2ad2add3b67dd3674d9af9dd91c1955a35d0e9",
]

In [13]:
censys_df.select("services.jarm.fingerprint_hex").show(3, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|fingerprint_hex                                                                                                                                               |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[26d26d16d26d26d22c26d26d26d26dfd9c9d14e4f4f67f94f0359f8b28f532]                                                                                              |
|[04d02d00004d04d04c04d02d04d04d92d68b02249d4ba2fa6016343a2accc4, null, null, null, null, 29d29d20d29d29d22c29d29d29d29df51c60823f963b0fe11deab8efc70b4b, null]|
|[null, 21d19d00021d21d00042d43d000000ee523fe586628a8d526db2ce24e29fb4]                                                                                        |
+---------------------------------

In [25]:
censys_df.filter(
    psf.array_contains(psf.col("services.jarm.fingerprint_hex"), win_server_2016_jarms[0])
).filter(
    psf.col("operating_system").isNotNull()
).groupBy("operating_system.product", "operating_system.version").count().sort("count", ascending=False).show(200, truncate=False)
#.groupBy("operating_system.edition").count().sort("count",ascending=False).show(truncate=False) # nothing!
#.select("operating_system.vendor", "operating_system.version", "operating_system.product", "operating_system.edition").show(truncate=False)

+-------------------------------------------+-------------------+------+
|product                                    |version            |count |
+-------------------------------------------+-------------------+------+
|Windows                                    |null               |358283|
|windows                                    |null               |256404|
|null                                       |null               |163782|
|linux                                      |null               |19838 |
|Linux                                      |null               |16629 |
|RouterOS                                   |null               |4867  |
|FortiOS                                    |null               |3896  |
|DSM                                        |null               |3535  |
|SonicOS                                    |null               |2478  |
|FreeBSD                                    |null               |2471  |
|VMware ESXi Server                         |null  

In [8]:
censys_df.printSchema()

root
 |-- host_identifier: struct (nullable = true)
 |    |-- ipv4: string (nullable = true)
 |    |-- ipv6: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- services: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- port: long (nullable = true)
 |    |    |-- transport: string (nullable = true)
 |    |    |-- service_name: string (nullable = true)
 |    |    |-- perspective: string (nullable = true)
 |    |    |-- observed_at: timestamp (nullable = true)
 |    |    |-- tls: struct (nullable = true)
 |    |    |    |-- version_selected: string (nullable = true)
 |    |    |    |-- cipher_selected: string (nullable = true)
 |    |    |    |-- certificates: struct (nullable = true)
 |    |    |    |    |-- leaf_fp_sha_256: binary (nullable = true)
 |    |    |    |    |-- chain_fps_sha_256: array (nullable = true)
 |    |    |    |    |    |-- element: binary (containsNull = true)
 |    |    |    |    |-- leaf_data: struct (null